# BrkRaw Python API to access PvDataset

In [1]:
import sys
import numpy as np
np.set_printoptions(precision=3, suppress=True)
import brkraw
import pprint as pp

In [2]:
path = '20190724_114946_BRKRAW_1_1.zip'
pvdset = brkraw.load(path)

## High-level API
### Printing out summary of dataset

In [3]:
pvdset.summary()

Paravision 6.0.1
----------------
UserAccount:	nmrsu
Date:		2019-07-24
Researcher:	BRKRAW
Subject ID:	BRKRAW
Session ID:	1
Study ID:	1
Date of Birth:	10 Jul 2019
Sex:		unknown
Weight:		0.202 kg
Subject Type:	Quadruped
Position:	Supine		Entry:	HeadFirst

[ScanID]	Sequence::Protocol::[Parameters]
[001]	Bruker:FLASH::0_Localizer_GOP::
	[ TR: 100 ms, TE: 2.5 ms, pixelBW: 292.96875 Hz, FlipAngle: 30 degree]
    [01] dim: 2D, matrix_size: 256 x 256 x 3, fov_size: 50 x 50 (unit:mm)
         spatial_resol: 0.195 x 0.195 x 1.000 (unit:mm), temporal_resol: 12800.000 (unit:msec)
[002]	Bruker:RARE::T2_TurboRARE_3D_20180423::
	[ TR: 1800 ms, TE: 34 ms, pixelBW: 685.30701754386 Hz, FlipAngle: 90 degree]
    [01] dim: 3D, matrix_size: 144 x 144 x 64, fov_size: 28.8 x 28.8 x 12.8 (unit:mm)
         spatial_resol: 0.200 x 0.200 x 0.200 (unit:mm), temporal_resol: 1134000.000 (unit:msec)
[003]	Bruker:EPI::2_BOLD_EPI_isotropic_wholebrain::
	[ TR: 2000 ms, TE: 14 ms, pixelBW: 3472.22222222222 Hz, FlipAngle

### Check session start time

In [4]:
pvdset.get_scan_time()

{'date': datetime.date(2019, 7, 24), 'start_time': datetime.time(11, 49, 46)}

### Check scan start time

In [5]:
visu_pars = pvdset.get_visu_pars(5, 1)
pvdset.get_scan_time(visu_pars)

{'date': datetime.date(2019, 7, 24),
 'start_time': datetime.time(11, 49, 46),
 'scan_time': datetime.time(13, 10, 49)}

### Specify scan id and reco id to access data object

In [6]:
scan_id = 5
reco_id = 1

### Printing out BIDS specific meta data

In [7]:
pvdset.print_bids(scan_id, reco_id)

Manufacturer:				Bruker BioSpin MRI GmbH
ManufacturersModelName:			Biospec 94/30
DeviceSerialNumber:			BT001005
StationName:				Biospec 94/30
SoftwareVersion:			6.0.1
MagneticFieldStrength:			9.403088884004417
ReceiveCoilName:			None
ReceiveCoilActiveElements:		None
GradientSetType:			None
MRTransmitCoilSequence:			{'Name': None, 'Manufacture': None, 'Type': None}
CoilConfigName:				None
MatrixCoilMode:				ParallelExperiment
CoilCombinationMethod:			None
PulseSequenceType:			EPI.ppg
ScanningSequence:			Bruker:EPI
SequenceVariant:			Both
ScanOptions:				{'RG': 'No', 'CG': 'No', 'PFF': 1, 'PFP': 1.4, 'FC': 'FlowNone', 'SP': 'On', 'FP': 'NoSuppression'}
SequenceName:				2_BOLD_EPI_isotropic_wholebrain
PulseSequenceDetails:			2_BOLD_EPI_isotropic_wholebrain (E6)
NonlinearGradientCorrection:		RectilinearTraversal
NumberShots:				1
ParallelReductionFactorInPlane:		1
ParallelAcquisitionTechnique:		None
PartialFourier:				1,1.4
PartialFourierDirection:		None
PhaseEncodingDirection:			j
Effecti

# BIDS parser syntax

#### Matadata Field Mapping for Bruker PvDataset
    - BIDS Metadata will be automatically created according to below reference.
    - If list is entered as value, each parameter will be tested and the first available value will be returned.
    - If dict is entered as value, below condition will be tested.
    - If key - where pair:  parse value from given key and return index of 'where' from these values
    - If key - idx pair:    parse value from given key and return value of given 'idx'
    - If 'Equation' in key: each key assigned as local variable and test in Equation will be executed to return the value
    - Else, new key - value dictionary will be return (for the cases with sub-keys)
    - If string is entered as value, The value of given parameter will be parsed from parameter files

In [8]:
# the default metadata reference.
pp.pprint(brkraw.lib.reference.METADATA_FILED_INFO)

{'AnatomicalLandmarkCoordinates': None,
 'CoilCombinationMethod': None,
 'CoilConfigName': 'ACQ_coil_config_file',
 'DeviceSerialNumber': 'VisuSystemOrderNumber',
 'DwellTime': {'BWhzPixel': 'VisuAcqPixelBandwidth', 'Equation': '1/BWhzPixel'},
 'EchoTime': 'VisuAcqEchoTime',
 'EffectiveEchoSpacing': {'ACCfactor': 'ACQ_phase_factor',
                          'BWhzPixel': 'VisuAcqPixelBandwidth',
                          'Equation': '(1 / (MatSizePE * BWhzPixel)) / '
                                      'ACCfactor',
                          'MatSizePE': {'idx': [{'key': 'VisuAcqGradEncoding',
                                                 'where': 'phase_enc'},
                                                {'key': 'VisuAcqImagePhaseEncDir',
                                                 'where': 'col_dir'}],
                                        'key': 'VisuCoreSize'}},
 'FlipAngle': 'VisuAcqFlipAngle',
 'GradientSetType': 'ACQ_status',
 'InstitutionAddress': None,
 'Institut

### Example of using custom metadata reference

In [9]:
metadata_ref = dict(ThisIsFlipAngle='VisuAcqFlipAngle',  # any parameter key among method, acqp, visupars can be used here
                    ThisIsJustTest='Im just string value',
                    
                    # If any operation required to set value, below example can be used.
                    ThisIsEquation=dict(a=3, 
                                        b=4, 
                                        Equation='a+b'),
                    
                    # Any standard and numpy method can be utilized to evaluate the value if needed.
                    Hierarchy=dict(SecondLevel='I am in the second level!',
                                   TryEquation=dict(abc='3',
                                                    Equation='list(range(abc))')),
                    
                    # Some parameter does not exists depends on the version of Paravision, this case
                    # You can use multiple keys to parse the first available value.
                    SequenceName = ['VisuAcquisitionProtocol', 'ACQ_protocol_name'],  
                    
                    # If no keys exists in parameter files, then you can set default string.
                    DefaultKeyIfNone = ['ImNotExistingKey', 'ImNeither', 'I am default value if nothing exists']
                   )

pvdset.print_bids(scan_id, reco_id, metadata=metadata_ref)

ThisIsFlipAngle:			70
ThisIsJustTest:				Im just string value
ThisIsEquation:				7
Hierarchy:				{'SecondLevel': 'I am in the second level!', 'TryEquation': [0, 1, 2]}
SequenceName:				2_BOLD_EPI_isotropic_wholebrain
DefaultKeyIfNone:			I am default value if nothing exists


In [10]:
visu_pars = pvdset.get_visu_pars(scan_id, reco_id)

In [11]:
# Some key contains multiple values
visu_pars.parameters['VisuAcqPartialFourier']

[1, 1.4]

In [12]:
# To parse only one value, you can use index
metadata_ref = dict(IHaveTwoValues='VisuAcqPartialFourier',
                    IWillTakeOnlyFirstValue=dict(key='VisuAcqPartialFourier',
                                                 idx=0),
                    ImCuriousTheIndexOfSpecificValue=dict(key='VisuAcqPartialFourier',
                                                 where=1.4))
pvdset.print_bids(scan_id, reco_id, metadata=metadata_ref)

IHaveTwoValues:				1,1.4
IWillTakeOnlyFirstValue:		1
ImCuriousTheIndexOfSpecificValue:	1


### Generate Your Own BIDS metadata reference for conversion set of data

- To use your own reference file for bids_converter function in command-line tool, please modify the json file generated by bids_list function. The Python API can be used for debuging purpose, and in case you want to implement your python dictionaty object instead of using file generated, dump your dictionary into JSON file as below

In [13]:
import json
with open('your_filename.json', 'w') as f:
    json.dump(metadata_ref, f, indent=4)

In [14]:
# The dumbed file will look like this.
with open('your_filename.json', 'r') as f:
    for l in f.readlines():
        print(l)

{

    "IHaveTwoValues": "VisuAcqPartialFourier",

    "IWillTakeOnlyFirstValue": {

        "key": "VisuAcqPartialFourier",

        "idx": 0

    },

    "ImCuriousTheIndexOfSpecificValue": {

        "key": "VisuAcqPartialFourier",

        "where": 1.4

    }

}


### Access Diffusion Direction for DTI image

In [15]:
pvdset.get_bdata(scan_id)  # bval, bvec, bmat will be returned (no example DTI file provided here)

(None, None, None)

### Access nibabel NifTi1Image object with header.

In [16]:
niiobj1 = pvdset.get_niftiobj(scan_id, reco_id)
print(niiobj1.affine, niiobj1.dataobj.shape)

[[  0.4    -0.      0.004 -14.381]
 [ -0.     -0.4     0.     14.968]
 [ -0.004   0.      0.4     1.824]
 [  0.      0.      0.      1.   ]] (72, 72, 32)


In [17]:
print(niiobj1.header)

<class 'nibabel.nifti1.Nifti1Header'> object, endian='<'
sizeof_hdr      : 348
data_type       : b''
db_name         : b''
extents         : 0
session_error   : 0
regular         : b''
dim_info        : 48
dim             : [ 3 72 72 32  1  1  1  1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
intent_code     : none
datatype        : int16
bitpix          : 16
slice_start     : 0
pixdim          : [-1.   0.4  0.4  0.4  2.   1.   1.   1. ]
vox_offset      : 0.0
scl_slope       : 0.017385513
scl_inter       : 0.0
slice_end       : 31
slice_code      : alternating increasing
xyzt_units      : 10
cal_max         : 0.0
cal_min         : 0.0
slice_duration  : 0.0625
toffset         : 0.0
glmax           : 0
glmin           : 0
descrip         : b''
aux_file        : b''
qform_code      : scanner
sform_code      : unknown
quatern_b       : 0.999986
quatern_c       : 0.0
quatern_d       : -0.005292898
qoffset_x       : -14.381493
qoffset_y       : 14.96818
qoffset_z       

### Access SimpleITK image object (the header is not preserved well compared to the nifti)

In [18]:
sitkobj = pvdset.get_sitkimg(scan_id, reco_id)
print(sitkobj.GetSpacing())
print(sitkobj.GetSize())
print(sitkobj.GetDirection())
print(sitkobj.GetOrigin())

(0.4, 0.4, 0.4)
(72, 72, 32)
(-0.999943970464059, 7.49879891330929e-33, -0.010585647480084857, 6.481839651694157e-19, 1.0, -6.122890913777527e-17, -0.010585647480084857, 6.123233995736766e-17, 0.999943970464059)
(14.3814929469549, -14.96818, 1.8244364794273555)


In [19]:
# Construct affine matrix from SimpleITK image object.
from nibabel.affines import from_matvec
resol = np.diag(sitkobj.GetSpacing())
direction = np.asarray(sitkobj.GetDirection())[:9].reshape([3, 3])
direction = resol.dot(direction)
origin = np.asarray(sitkobj.GetOrigin())[:3]
affine_from_sitk = from_matvec(direction, origin)
affine_from_sitk = np.diag([-1, -1, 1, 1]).dot(affine_from_sitk)
print(affine_from_sitk)

[[  0.4    -0.      0.004 -14.381]
 [ -0.     -0.4     0.     14.968]
 [ -0.004   0.      0.4     1.824]
 [  0.      0.      0.      1.   ]]


In [ ]:
# export nifti data
pvdset.save_as(scan_id, reco_id, 'your_filename', 
               dir='./', ext='nii.gz', 
               crop=[10, 100])  # you can crop the data thorugh this option


# export diffution parameters !! This will only function when image has diffusion direction in their header, 
# the example data does not have it.
pvdset.save_bdata(scan_id, 'your_filename', 
                  dir='./')

# export BIDS json file with your metadata_ref
pvdset.save_json(scan_id, reco_id, 'your_filename', 
                 dir='./', metadata=metadata_ref)

## Low-level API

### load JCAMP-DX parameter objects
- Using the methods below, each parameter can be accessed as dictionary object. and the value is converted to python compatible type (some value may not converted due to the complex structure)

In [21]:
acqp        = pvdset.get_acqp(scan_id)
method      = pvdset.get_method(scan_id)
visu_pars   = pvdset.get_visu_pars(scan_id, reco_id)

In [22]:
pvdset._subject.parameters

OrderedDict([('SUBJECT_version_nr', 1),
             ('SUBJECT_id', 'BRKRAW'),
             ('SUBJECT_instance_uid',
              '2.16.756.5.5.100.8323328.194975.1563983386.19519'),
             ('SUBJECT_instance_creation_date', [[1563983386, 824, -240]]),
             ('SUBJECT_name_string', 'BRKRAW'),
             ('SUBJECT_name', [[1, None]]),
             ('SUBJECT_dbirth', '10 Jul 2019'),
             ('SUBJECT_type', 'Quadruped'),
             ('SUBJECT_sex', 'unknown'),
             ('SUBJECT_sex_animal', 'UNKNOWN'),
             ('SUBJECT_remarks', 'VIRGIN RAT'),
             ('SUBJECT_study_name', 1),
             ('SUBJECT_study_nr', 1),
             ('SUBJECT_study_instance_uid',
              '2.16.756.5.5.100.8323328.194975.1563983386.19520'),
             ('SUBJECT_weight', 0.202),
             ('SUBJECT_referral', 'nmrsu'),
             ('SUBJECT_purpose', None),
             ('SUBJECT_pos_valid', 'Yes'),
             ('SUBJECT_entry', 'SUBJ_ENTRY_HeadFirst'),
       

- All parameter object has attibutes of headers and parameters

In [23]:
acqp.headers

OrderedDict([('TITLE', 'Parameter List, ParaVision 6.0.1'),
             ('JCAMPDX', 4.24),
             ('DATATYPE', 'Parameter Values'),
             ('ORIGIN', 'Bruker BioSpin MRI GmbH'),
             ('OWNER', 'nmrsu')])

In [24]:
acqp.parameters

OrderedDict([('PULPROG', 'EPI.ppg'),
             ('GRDPROG', None),
             ('ACQ_experiment_mode', 'ParallelExperiment'),
             ('ACQ_ReceiverSelect', ['Yes', 'Yes', 'Yes', 'Yes']),
             ('ACQ_PexSelect', 0),
             ('ACQ_load_shim_set_list', 'No'),
             ('ACQ_user_filter', 'No'),
             ('ACQ_DS_enabled', 'No'),
             ('ACQ_switch_pll_enabled', 'No'),
             ('ACQ_preload', 0),
             ('ACQ_branch_preload',
              array([[1000000, 1000000],
                     [1000000, 1000000],
                     [1000000, 1000000],
                     [1000000, 1000000],
                     [1000000, 1000000],
                     [1000000, 1000000],
                     [1000000, 1000000],
                     [1000000, 1000000],
                     [1000000, 1000000],
                     [    500,     500]])),
             ('ACQ_jobs_size', 0),
             ('ACQ_jobs', [[0, 1, 0, 1, 64, 100, 0, 0]]),
             ('ACQ_di

In [25]:
method.headers

OrderedDict([('TITLE', 'Parameter List, ParaVision 6.0.1'),
             ('JCAMPDX', 4.24),
             ('DATATYPE', 'Parameter Values'),
             ('ORIGIN', 'Bruker BioSpin MRI GmbH'),
             ('OWNER', 'nmrsu')])

In [26]:
method.parameters

OrderedDict([('Method', 'Bruker:EPI'),
             ('PVM_EffSWh', 250000),
             ('EchoTime', 14),
             ('PVM_MinEchoTime', 8.324),
             ('PVM_EchoTime', 0.288),
             ('NSegments', 1),
             ('PVM_RepetitionTime', 2000),
             ('PackDel', 0),
             ('PVM_NEchoImages', 1),
             ('PVM_NAverages', 1),
             ('PVM_NRepetitions', 1),
             ('PVM_ScanTimeStr', '0h0m2s0ms'),
             ('PVM_ScanTime', 2000),
             ('PVM_SignalType', 'SignalType_Fid'),
             ('PVM_DeriveGains', 'Yes'),
             ('PVM_SpatDimEnum', '2D'),
             ('PVM_IsotropicFovRes', 'Isot_None'),
             ('PVM_Matrix', [72, 72]),
             ('PVM_MinMatrix', [16, 16]),
             ('PVM_MaxMatrix', [512, 512]),
             ('PVM_DefMatrix', [96, 64, 32]),
             ('PVM_AntiAlias', [1, 1]),
             ('PVM_MaxAntiAlias', [2, 8]),
             ('PVM_EncSpectroscopy', 'No'),
             ('PVM_EncUseMultiRec', 

In [27]:
visu_pars.headers

OrderedDict([('TITLE', 'Parameter List, ParaVision 6.0.1'),
             ('JCAMPDX', 4.24),
             ('DATATYPE', 'Parameter Values'),
             ('ORIGIN', 'Bruker BioSpin MRI GmbH'),
             ('OWNER', 'nmrsu')])

In [28]:
visu_pars.parameters

OrderedDict([('VisuVersion', 3),
             ('VisuUid', '2.16.756.5.5.100.8323328.5482.1563988249.1'),
             ('VisuCreator', 'ParaVision'),
             ('VisuCreatorVersion', '6.0.1'),
             ('VisuCreationDate', ['2019-07-24T13:10:49', '772-0400']),
             ('VisuInstanceModality', 'MR'),
             ('VisuCoreFrameCount', 32),
             ('VisuCoreDim', 2),
             ('VisuCoreSize', [72, 72]),
             ('VisuCoreDimDesc', ['spatial', 'spatial']),
             ('VisuCoreExtent', [28.8, 28.8]),
             ('VisuCoreFrameThickness', 0.4),
             ('VisuCoreUnits', ['mm', 'mm']),
             ('VisuCoreOrientation',
              array([[ 1.   ,  0.011,  0.   ,  0.   ,  0.   , -1.   , -0.011,  1.   ,
                       0.   ],
                     [ 1.   ,  0.011,  0.   ,  0.   ,  0.   , -1.   , -0.011,  1.   ,
                       0.   ],
                     [ 1.   ,  0.011,  0.   ,  0.   ,  0.   , -1.   , -0.011,  1.   ,
                   

### Useful method to get parameters for data reconstruction and conversion.
- All function listed above is private methods

In [29]:
pvdset._get_dim_info(visu_pars)

(2, 'spatial_only')

In [30]:
pvdset._get_gradient_encoding_info(visu_pars)

['read_enc', 'phase_enc']

In [31]:
pvdset._get_frame_group_info(visu_pars)

{'frame_type': 'MAGNITUDE_IMAGE',
 'frame_size': 32,
 'matrix_shape': [32],
 'group_id': ['FG_SLICE'],
 'group_comment': [None],
 'dependent_vals': [[['VisuCoreOrientation', 0], ['VisuCorePosition', 0]]]}

In [32]:
pvdset._get_slice_info(visu_pars)

{'num_slice_packs': 1,
 'num_slices_each_pack': [32],
 'slice_distances_each_pack': [0.4],
 'unit_slice_distances': 'mm'}

In [33]:
pvdset._get_spatial_info(visu_pars)

{'spatial_resol': [(0.4, 0.4, 0.4)],
 'matrix_size': [(72, 72, 32)],
 'fov_size': [28.8, 28.8],
 'unit': 'mm'}

In [34]:
pvdset._get_temp_info(visu_pars)

{'temporal_resol': 2000.0, 'num_frames': 1, 'unit': 'msec'}

In [35]:
pvdset._get_orient_info(visu_pars, method)

{'subject_type': 'Quadruped',
 'subject_position': 'Head_Supine',
 'volume_position': array([-14.381,  -1.824,  14.968]),
 'orient_matrix': array([[ 1.   ,  0.011,  0.   ],
        [ 0.   ,  0.   , -1.   ],
        [-0.011,  1.   ,  0.   ]]),
 'orient_order': [0, 2, 1],
 'gradient_orient': array([[[ 1.   ,  0.011,  0.   ],
         [ 0.   ,  0.   ,  1.   ],
         [-0.011,  1.   ,  0.   ]]])}

In [36]:
pvdset._get_disk_slice_order(visu_pars)

'normal'

### Access dataobj

In [37]:
dataobj     = pvdset.get_dataobj(scan_id, reco_id, slp_correct=False)     
matrix_size = pvdset.get_matrix_size(scan_id, reco_id)  
affine      = pvdset.get_affine(scan_id, reco_id)   

In [38]:
print(dataobj.shape, matrix_size)

(72, 72, 32) [72, 72, 32]


In [39]:
print(affine)

[[  0.4    -0.      0.004 -14.381]
 [ -0.     -0.4     0.     14.968]
 [ -0.004   0.      0.4     1.824]
 [  0.      0.      0.      1.   ]]


In [40]:
dataobj.dtype

dtype('int16')

- If slp_correct option is False, the dataobj returned the rescaled int16 data, 
- If slp_correct option is True, below step will performed internally

In [41]:
from brkraw.lib.utils import is_all_element_same
data_slope = visu_pars.parameters['VisuCoreDataSlope']
data_offset = visu_pars.parameters['VisuCoreDataOffs']

In [42]:
print(is_all_element_same(data_slope), # check if all elements are same.
      is_all_element_same(data_offset)
     )

True True


In [45]:
# correcting intensity to original value
orj_dataobj = dataobj * data_slope[0] + data_offset[0]
print(orj_dataobj.dtype)

# build nibabel object
import nibabel as nib
niiobj2 = nib.Nifti1Image(orj_dataobj, affine)

float64


### Access binary FID file obj

- The FID handling is currently experimental, only showing here to present potential application.

In [46]:
from brkraw.lib.reference import WORDTYPE
print(acqp.parameters['GO_raw_data_format'])

dt_code = WORDTYPE[acqp.parameters['GO_raw_data_format'][2:]]
print(dt_code)

fid_binary = pvdset.get_fid(scan_id)

# convert to numpy array
fid = np.frombuffer(fid_binary, dt_code)

GO_32BIT_SGN_INT
i


In [47]:
fid.shape

(940032,)